### План работы на второй спринт (25/01/25-07/02/25)
**Модель:** Логистическая Регрессия  
**Датасет:** [Titanic](https://www.kaggle.com/datasets/yasserh/titanic-dataset)

1. **Присоединить данные**  
   Загрузить данные из источника, организовать их в DataFrame.   

2. **Изучить датасет**  
   - Определить назначение колонок.  
   - Использовать .info(), .describe(), .head() и др. для анализа структуры данных и типов признаков.  
   -Построить пару графиков. Скажем, оценить распределение данных, например возраста. Или показать зависимость между ключевыми параметрами (для вдохновения  смотри appendix).

3. **Почистить данные**  
   - Оценить наличие пропущенных данных (.isnull().sum()).
   - Если нужно, обработать их:
   - Удаление строк/столбцов с большим количеством пропусков.
   - (*) Или заполнение медианой, средним, модой.  
   - (**) Заменить категориальные данные числовыми (если необходимо). Использовать стандартные средства питона или, например, используя OneHotEncoder  
   - (***) Применить StandardScaler для числовых признаков.

4. **Создать базовую модель**  
   - Выбрать входные параметры (X) и целевой параметр (y). 
   - Разделить данные на тренировочный и тестовый наборы.
   - Построить модель логистической регрессии на основе X и y.  
   


In [ ]:
"""
# 📝 Задание: Построение полносвязной нейронной сети (MLP) для классификации рукописных цифр

## **Описание задания**
В этом задании вам предстоит построить **полносвязную нейронную сеть (MLP)** для классификации изображений рукописных цифр из датасета **MNIST**.

### **Что вы изучите?**
- Как загружать и предобрабатывать данные MNIST.
- Как строить многослойную нейросеть на Keras.
- Как обучать и тестировать модель.
- Как визуализировать результаты работы нейросети.
- Как изменять архитектуру модели и анализировать её влияние на качество классификации.

## **Структура задания**
1. **Загрузка данных** MNIST.
2. **Предобработка** (нормализация, one-hot encoding).
3. **Построение нейросети** (полносвязная архитектура с Keras).
4. **Изменение архитектуры модели** (добавление новых слоев, изменение количества нейронов).
5. **Подбор количества эпох** (эксперименты с количеством эпох для поиска оптимального значения).
6. **Анализ точности модели** и **поиск цифры с наибольшими и наименьшими ошибками**.
7. **Разбор терминов** (adam, categorical_crossentropy, bias, loss function, softmax).
8. **Попытка объяснить, что такое перцептрон** окружающим.

"""

# 📌 Шаг 1: Импорт необходимых библиотек
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten

# 📌 Шаг 2: Загрузка данных MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 📌 Шаг 3: Предобработка данных
# Нормализация изображений (0-255 → 0-1)
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encoding меток классов
y_train_cat = keras.utils.to_categorical(y_train, 10)
y_test_cat = keras.utils.to_categorical(y_test, 10)

# 📌 Шаг 4: Визуализация первых 25 изображений
plt.figure(figsize=(10, 5))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i], cmap=plt.cm.binary)
plt.show()

# 📌 Шаг 5: Создание базовой модели MLP
model = keras.Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Вывод структуры модели
print(model.summary())

# 📌 Шаг 6: Изменение архитектуры модели
# 🛠️ **Задание**: Добавьте дополнительные слои или измените количество нейронов в скрытом слое.
# Пример: Добавление второго скрытого слоя
model = keras.Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])
print(model.summary())

# 📌 Шаг 7: Подбор количества эпох
# 🛠️ **Задание**: Измените количество эпох обучения и найдите оптимальное значение.
epochs_list = [5, 10, 20]
results = {}

for epochs in epochs_list:
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train_cat, batch_size=32, epochs=epochs, validation_split=0.2)
    loss, accuracy = model.evaluate(x_test, y_test_cat)
    results[epochs] = accuracy

# 📌 Шаг 8: Анализ точности модели
print("Результаты точности модели для разного количества эпох:")
for epochs, acc in results.items():
    print(f"Эпохи: {epochs}, Точность: {acc:.4f}")

# 📌 Шаг 9: Поиск цифры с наибольшими и наименьшими ошибками
predictions = model.predict(x_test)
pred_labels = np.argmax(predictions, axis=1)
incorrect = pred_labels != y_test

unique, counts = np.unique(y_test[incorrect], return_counts=True)
error_analysis = dict(zip(unique, counts))
most_errors = max(error_analysis, key=error_analysis.get)
least_errors = min(error_analysis, key=error_analysis.get)

print(f'Больше всего ошибок на цифре: {most_errors}')
print(f'Меньше всего ошибок на цифре: {least_errors}')

# 📌 Шаг 10: Разбор терминов
terms = {
    "Adam": "Оптимизатор, адаптивно изменяющий скорость обучения.",
    "categorical_crossentropy": "Функция потерь для многоклассовой классификации.",
    "bias": "Смещение (b) в нейроне, помогает сдвигать функцию активации.",
    "loss function": "Функция, определяющая разницу между предсказанием и истинными значениями.",
    "softmax": "Активационная функция, превращающая выходные значения в вероятности."
}
for term, definition in terms.items():
    print(f"{term}: {definition}")

# 📌 Шаг 11: Объяснение перцептрона
# 🛠️ **Задание**: Найдите кого-то, кто не знаком с нейросетями, и попробуйте объяснить ему, что такое перцептрон. Напишите, как это прошло.
